In [ ]:
class LinearRegressionScratch:
  def __init__ (self, learning_rate=0.001,epochs = 1000,verbose = False):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.verbose = verbose
    self.weights = None
    self.bias = None

  def fit(self,X,y):
    num_samples = X.shape[0]
    num_features = X.shape[1]
    self.weights = np.random.rand(num_features)
    self.bias = 0

    for epoch in range(self.epochs):
      y_preds = X.dot(self.weights) + self.bias
      dw = 2/num_samples * X.T.dot(y_preds-y)
      db = 2/num_samples * np.sum(y_preds-y)
      self.weights -= dw * self.learning_rate
      self.bias -= db * self.learning_rate

      if epoch%100 == 0 and self.verbose:
        loss = np.mean((y-y_preds)**2)
        print(f"epoch {epoch} = {loss}")


  def predict(self,X):
    y_preds = X.dot(self.weights) + self.bias
    return y_preds


In [ ]:
import numpy as np

# Create synthetic data: y = 4 + 3x + noise
np.random.seed(24)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# Flatten y to 1D array for simplicity
y = y.flatten()

# Instantiate and train the model
model = LinearRegressionScratch(learning_rate=0.01, epochs=1000,verbose = True)
model.fit(X, y)

# Make predictions on the training data
predictions = model.predict(X)

# Print the first 5 predictions and actual values
print("Predictions:", predictions[:5])
print("Actual:", y[:5])


epoch 0 = 47.645564339829946
epoch 100 = 1.1601923521611341
epoch 200 = 1.0587888779456276
epoch 300 = 1.0019956630000397
epoch 400 = 0.9694164817709912
epoch 500 = 0.9507275349601905
epoch 600 = 0.940006678272282
epoch 700 = 0.9338566919040696
epoch 800 = 0.9303287711949103
epoch 900 = 0.9283049903627374
Predictions: [ 9.86023544  8.26043754 10.1049596   5.31610264  6.18193545]
Actual: [ 9.61267695  9.75837997 10.68391375  5.34986616  5.3138632 ]


In [ ]:
class LogisticRegressionScratch:
  def __init__(self,learning_rate = 0.001, epochs = 1000,verbose = False):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.verbose = verbose
    self.weights = None
    self.bias = None

  def sigmoid(self,X):
    return 1/(1+np.exp(-X))

  def fit(self,X,y):
    num_samples = X.shape[0]
    num_features = X.shape[1]
    self.weights = np.random.randn(num_features)
    self.bias = 0

    for epoch in range(self.epochs):
      z = X.dot(self.weights) + self.bias
      y_preds = self.sigmoid(z)
      dw = 1/num_samples * X.T.dot(y_preds-y)
      db = 1/num_samples * np.sum(y_preds-y)

      self.weights -= dw * self.learning_rate
      self.bias -= db * self.learning_rate

      if self.verbose and epoch % 100 == 0:
        epsilon = 1e-15
        loss = - 1 / num_samples * np.sum(y * np.log(y_preds+epsilon) + (1-y) * np.log(1-y_preds+epsilon))
        print(f"epoch {epoch} loss = {loss}")

  def predict(self,X):
    z = X.dot(self.weights) + self.bias
    y_preds = self.sigmoid(z)
    return (y_preds>=0.5).astype(int)

  def accuracy(self,X,y):
    y_preds = self.predict(X)
    return np.mean(y_preds == y)


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate dummy binary classification data
X, y = make_classification(n_samples=500, n_features=5, n_informative=3, n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegressionScratch(learning_rate=0.1, epochs=1000,verbose=True)
model.fit(X_train, y_train)
print("Train Accuracy:", model.accuracy(X_train, y_train))
print("Test Accuracy:", model.accuracy(X_test, y_test))

epoch 0 loss = 1.2280363329325863
epoch 100 loss = 0.31134343648661256
epoch 200 loss = 0.29804721038330917
epoch 300 loss = 0.2942835737137249
epoch 400 loss = 0.29286779622857684
epoch 500 loss = 0.29226968728198666
epoch 600 loss = 0.2920005917412164
epoch 700 loss = 0.29187483679695864
epoch 800 loss = 0.29181462328936186
epoch 900 loss = 0.291785323903164
Train Accuracy: 0.885
Test Accuracy: 0.9


In [ ]:
class Node:
  def __init__(self, feature_index = None, threshhold=None,left=None,right=None,value=None):
    self.feature_index = feature_index
    self.threshhold=threshhold
    self.left = left
    self.right = right
    self.value = value

class DecisionTreeScratch:
  def __init__(self, max_depth = 5,min_samples_split = 5):
    self.root = None
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split

  def fit(self,X,y):
    self.root = self.build_tree(X,y,depth = 0)

  def build_tree(self, X, y, depth):
      if len(X) <= self.min_samples_split or len(np.unique(y)) == 1 or depth == self.max_depth:
          leaf_value = self._most_common(y)
          return Node(value=leaf_value)

      parent_entropy = self._entropy(y)

      best_ig = -float('inf')
      best_feature_index = None
      best_threshold = None
      best_left_X, best_left_y = None, None
      best_right_X, best_right_y = None, None

      for feature_index in range(X.shape[1]):
          feature_values = np.sort(np.unique(X[:, feature_index])) #gets all unique features in this current feature and sorts them
          thresholds = (feature_values[:-1] + feature_values[1:]) / 2 #threshholds are between each unique point

          for threshold in thresholds:
              left_mask = X[:, feature_index] <= threshold #splits X into groups based on threshhold
              right_mask = X[:, feature_index] > threshold
              X_left, y_left = X[left_mask], y[left_mask]
              X_right, y_right = X[right_mask], y[right_mask]

              if len(y_left) == 0 or len(y_right) == 0:
                  continue  # Skip invalid splits

              weighted_entropy = (len(y_left) / len(y)) * self._entropy(y_left) + (len(y_right) / len(y)) * self._entropy(y_right)
              information_gain = parent_entropy - weighted_entropy

              if information_gain > best_ig:
                  best_ig = information_gain
                  best_feature_index = feature_index
                  best_threshold = threshold
                  best_left_X, best_left_y = X_left, y_left
                  best_right_X, best_right_y = X_right, y_right

      # If no good split found, make a leaf
      if best_ig == -float('inf'):
          leaf_value = self._most_common(y)
          return Node(value=leaf_value)

      # Recursively build left and right subtrees
      left_child = self.build_tree(best_left_X, best_left_y, depth + 1)
      right_child = self.build_tree(best_right_X, best_right_y, depth + 1)

      # Return a decision node
      return Node(feature_index=best_feature_index, threshhold=best_threshold, left=left_child, right=right_child)




  def predict(self,X):
    return np.array([self._traverse_tree(x, self.root) for x in X])


  def _traverse_tree(self, x, node):
      if node.value is not None:
          return node.value

      feature_value = x[node.feature_index]
      if feature_value <= node.threshhold:
        return self._traverse_tree(x,node.left)
      else:
        return self._traverse_tree(x,node.right)



  def _entropy(self,y):
    values,counts = np.unique(y,return_counts=True)
    probabilities = counts/len(y)
    entropy = - np.sum(probabilities * np.log2(probabilities))
    return entropy

  def _most_common(self,y):
    value,counts = np.unique(y,return_counts=True)
    return value[np.argmax(counts)]

  def score(self,X,y):
    y_preds = self.predict(X)
    return np.mean(y_preds==y)



In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load Iris data
iris = load_iris()
X, y = iris.data, iris.target


# For simplicity, let's do a binary classification: class 0 vs not 0
y_binary = (y == 0).astype(int)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)


tree = DecisionTreeScratch(max_depth=3, min_samples_split=5)
tree.fit(X_train, y_train)

print("Train accuracy:", tree.score(X_train, y_train))
print("Test accuracy:", tree.score(X_test, y_test))


Train accuracy: 1.0
Test accuracy: 1.0


In [ ]:
class RandomForestScratch: #literally just a bunch of decision trees predict on it then get majority vote XD
  def __init__(self, max_depth = 5,min_samples_split = 5,num_trees=10):
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.num_trees = num_trees
    self.trees = []


  def _bootstrap_samples(self,X,y): #get samples using replacement
    num_samples = X.shape[0]
    indices = np.random.choice(num_samples,size=num_samples,replace=True)
    X_bootstrap = X[indices]
    y_bootstrap = y[indices]
    return X_bootstrap,y_bootstrap

  def fit(self,X,y):
    for i in range(self.num_trees):
      X_bootstrap,y_bootstrap = self._bootstrap_samples(X,y)
      new_tree = DecisionTreeScratch(self.max_depth,self.min_samples_split)
      new_tree.fit(X_bootstrap,y_bootstrap)
      self.trees.append(new_tree)

  def predict(self,X):
    all_preds = []

    for tree in self.trees:
        preds = tree.predict(X)
        all_preds.append(preds)

    all_preds = np.array(all_preds)
    all_preds = all_preds.T
    final_preds = [np.bincount(row).argmax() for row in all_preds]
    return np.array(final_preds)





In [ ]:
import numpy as np


# Class 0: centered at (0,0)
X_class0 = np.random.randn(50, 2) * 0.5 + np.array([0, 0])

# Class 1: centered at (2,2)
X_class1 = np.random.randn(50, 2) * 0.5 + np.array([2, 2])

X_train = np.vstack((X_class0, X_class1))
y_train = np.array([0]*50 + [1]*50)

indices = np.arange(len(y_train))
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices]

X_test = np.array([
    [0, 0],    # close to class 0
    [2, 2],    # close to class 1
    [1, 1],    # in between
    [3, 3],    # far class 1
    [-1, -1]   # far class 0
])

# Create, train and predict
rf = RandomForestScratch(max_depth=5, min_samples_split=2, num_trees=10)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Test predictions:", y_pred)


Test predictions: [0 1 1 1 0]


In [ ]:
from collections import Counter


class knnScratch:
  def __init__(self,k):
    self.k = k

  def fit(self,X,y):
    self.X_train = X
    self.y_train = y

  def predict(self, x):
      diff = self.X_train - x  # broadcast subtraction
      distances = np.sum(diff ** 2, axis=1)  # squared Euclidean distances for all training samples
      k_indices = np.argsort(distances)[:self.k]

      # Get the labels of those k nearest neighbors
      k_labels = self.y_train[k_indices]

      majority_label = Counter(k_labels).most_common(1)[0][0]
      return majority_label


  def predict_batch(self, X):
      return [self.predict(x) for x in X]

In [ ]:
import numpy as np

X_train = np.array([[1,2], [2,3], [3,4], [5,5]])
y_train = np.array([0, 0, 1, 1])

model = knnScratch(k=3)
model.fit(X_train, y_train)

X_test = np.array([[2,2], [4,4]])
predictions = model.predict_batch(X_test)
print(predictions)


[np.int64(0), np.int64(1)]
